1. 라벨 인코딩할 때 원본/숫자 어떻게 변한건지 확인할 수 있도록

# 0. GPU check

* 이 코드는 Nvidia GPU를 사용하는 컴퓨터에서, train / test 데이터가 분리되어있는 csv 파일을 사용하는 것을 전제로 작성됨

In [1]:
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print("device_count: {}".format(device_count))
    for device_num in range(device_count):
        print("device {} capability {}".format(
            device_num,
            torch.cuda.get_device_capability(device_num)))
        print("device {} name {}".format(
            device_num, 
            torch.cuda.get_device_name(device_num)))
else:
    print("no cuda device")

device_count: 1
device 0 capability (8, 6)
device 0 name NVIDIA GeForce RTX 3080


In [2]:
if torch.cuda.is_available() :
    device = torch.device("cuda:0")
else : 
    device = torch.device("cpu")

In [3]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    
print_gpu_utilization()

GPU memory occupied: 7505 MB.


* 모델 훈련과정에서 GPU 메모리 용량 초과 시, 개발서버 콘솔에서 직접 `nvidia-smi` 명령어 실행 후 메모리를 점유하고 있는 process의 PID를 찾아 `sudo kill -9 {pid}` 로 프로세스 종료해주면 됨

# 1. Import packages

In [4]:
## Need to check if packages are compatible
# !pip install accelerate nvidia-ml-py3
# !pip install datasets==2.4.0
# !pip install huggingface_hub==0.9.1
# !pip install transformers==4.22.1 # bf16, tf32 등 사용하려면 4.2 이상 필요
# !pip install pyarrow==9.0.0

* huggingface_hub와 transformers 간 호환가능한 버전 확인 필요
* 만약 성능 테스트를 위해 datasets api를 사용할거라면 datasets 역시 호환 가능 버전 확인해야 함
* 세 가지 dependencies를 사용한다는 가정 하에, pyarrow 라이브러리도 필요.

In [5]:
import transformers
import datasets
import huggingface_hub
import pyarrow

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(pyarrow.__version__)

# 4.22.1
# 2.4.0
# 0.9.1
# 9.0.0

4.22.1
2.4.0
0.9.1
9.0.0


In [6]:
import os
import re
import math
import numpy as np
import pandas as pd

# 'You can use tf32' if you are acessing Ampere hardware
import torch
torch.backends.cuda.matmul.allow_tf32 = True

from datasets import load_dataset, load_metric, ClassLabel
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.examples.pbt_transformers.utils import (
    download_data,
    build_compute_metrics_fn,
)
from ray.tune.schedulers import PopulationBasedTraining
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
    EarlyStoppingCallback
)

# 2. Import Data

* xxx_train.csv, xxx_test.csv 파일은 아래 형식으로 전처리된 csv 파일이어야 함 (column name: `text`, `label`)


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600", style="text-align:center">
    text
    </th>
    <th class="mdc-text-purple-600", style="text-align:center">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Ok lar... Joking wif u oni...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      U dun say so early hor... U c already then say...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Nah I don't think he goes to usf, he lives around here though
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
</table>

In [7]:
data_name = "naver_movie_review" ## IMDB / naver_movie_review / spam

dataset = load_dataset('csv', data_files={'train': f'../data_splited/{data_name}_train.csv',
                                          'test': f'../data_splited/{data_name}_test.csv'})
dataset

Using custom data configuration default-e7166b2526575299
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-e7166b2526575299/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

# 3. Data Preprocessing

* load_dataset 함수로 불러온 데이터를 수정할 때는 수정 내용을 담은 함수를 만들고, 이를 map 함수로 각 원소에 적용함 ([링크](https://huggingface.co/docs/datasets/v1.4.0/processing.html#processing-data-row-by-row)에서 확인)

In [8]:
## remove specal characters

def remove_sp(example):
    example["text"]=re.sub(r'[^a-z|A-Z|0-9|ㄱ-ㅎ|ㅏ-ㅣ|가-힣| ]+', '', str(example["text"]))
    return example

dataset = dataset.map(remove_sp)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e7166b2526575299/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-b43d4ea7484dfb3f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e7166b2526575299/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-61842ae510046f1a.arrow


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [10]:
## label encoding

labels = list(set(dataset["train"]["label"] + dataset["test"]["label"]))
num_labels = len(labels)

def encoding_label(example):
    str_to_int = ClassLabel(num_classes=num_labels, names=labels)
    example["label"]=str_to_int.str2int(example["label"])
    return example

if type(labels[0]) == str:
    dataset = dataset.map(encoding_label)
    
print(num_labels)

2


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [12]:
# For IMDB and Naver Movie Review, 

# Make imbalanced data to test model performance (label 0:label 1 = 8:2)
# https://discuss.huggingface.co/t/huggingface-datasets-convert-a-dataset-to-pandas-and-then-convert-it-back/14708/3

# df_train = pd.DataFrame(dataset['train'])
# df_train_0 = df_train[df_train["label"]==0]
# df_train_1 = df_train[df_train["label"]==1].sample(frac=1)[0:math.floor(len(df_train[df_train['label']==0])*0.2)]
# dataset["train"] = datasets.Dataset.from_pandas(pd.concat([df_train_0,df_train_1]), preserve_index=False)
# dataset

# 4. Load PLM & Tokenizing

In [13]:
# model_name = "bert-base-cased"
# model_name = "klue/bert-base"

# model_name = "bert-base-multilingual-cased"

# model_name = "xlm-roberta-base"
model_name = "klue/roberta-base"

In [14]:
# Download cache tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [15]:
def tokenize_function(examples):
    tokenized_batch = tokenizer(examples["text"], padding="max_length", truncation=True) # padding : ['longest', 'max_length', 'do_not_pad']
    return tokenized_batch

In [16]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [17]:
# train_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(0,math.floor(len(tokenized_datasets["train"])*0.7)))
# eval_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(math.floor(len(tokenized_datasets["train"])*0.7), len(tokenized_datasets["train"])))
# test_dataset = tokenized_datasets["test"]

In [18]:
# data for test
train_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(1000))
eval_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(1000))
test_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-e7166b2526575299/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-a7743211b41014c6.arrow


# 5. Check class weights

In [19]:
def class_weight(train_dataset) :
    
    train_labels = np.array(train_dataset["label"])
    class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)
    
    weights = torch.tensor(class_weights, dtype = torch.float)
    
    return weights

In [20]:
weights = class_weight(train_dataset)
print(weights)

tensor([1.0638, 0.9434])


# 6. Modeling

In [21]:
## Customize training strategy

task_data_dir = "test-model"
gpus_per_trial = 1
cpus_per_trial = 16
n_trials = 5
seed = 818

In [22]:
# Download model and features

config = AutoConfig.from_pretrained(
    model_name, 
    num_labels=num_labels
)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        config=config
        )

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

In [23]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc") # Accuracy/F1
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [24]:
training_args = TrainingArguments(
    output_dir=".",
    learning_rate=2e-5, # config
    do_train=True,
    do_eval=True,
    no_cuda=gpus_per_trial <= 0,
    evaluation_strategy="steps",
    save_strategy="steps",
    metric_for_best_model="f1",
    greater_is_better=True,
    load_best_model_at_end=True,
    num_train_epochs=2,  # config
    max_steps=-1,  # config
    per_device_train_batch_size=8,  # config
    per_device_eval_batch_size=8,  # config
    warmup_steps=0,
    warmup_ratio=0.1,  # config
    weight_decay=0.1,  # config
    logging_dir="./logs",
    skip_memory_metrics=True,
    report_to="none",
    fp16=True,
    # bf16=True,
    # tf32=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    seed=seed,
    eval_steps = 50
    )
    
# trainer = Trainer(
#     model_init=model_init,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     compute_metrics=compute_metrics,
#     )

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        weight = weights.to(device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    
trainer = CustomTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--klue--roberta-base/snapshots/67dd433d36ebc66a42c9aaa85abcf8d2620e41d9/pytorch_model.bin
Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequ

In [26]:
# Hyperparameter tuning with ray tune

tune_config = {
    "num_train_epochs": tune.choice([2, 5]),
}

# PopulationBasedTraining
# worker might copy the model parameters from a better performing worker or explore new hyperparameters by changing the current values randomly
# cf. ASHAScheduler
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="eval_f1",
    mode="max",
    perturbation_interval=1,
    hyperparam_mutations={
        "weight_decay": tune.uniform(0.0, 0.3), # tune.uniform(1, 10) == np.random.uniform(1, 10)
        "learning_rate": tune.uniform(1e-5, 5e-5),
        "warmup_ratio": tune.uniform(0.0, 0.3),
    },
)


reporter = CLIReporter(
    parameter_columns={
        "weight_decay": "w_decay",
        "learning_rate": "lr",
        "per_device_train_batch_size": "train_bs/gpu",
        "num_train_epochs": "num_epochs",
    },
    metric_columns=["eval_f1", "eval_accuracy", "eval_loss", "epoch", "training_iteration"],
)

result = trainer.hyperparameter_search(
    direction = "maximize",
    hp_space = lambda _: tune_config,
    backend="ray",
    n_trials=n_trials,
    resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
    scheduler=scheduler,
    keep_checkpoints_num=1,
    checkpoint_score_attr="training_iteration",
    stop=None,
    progress_reporter=reporter,
    local_dir="./test-results",
    name="tune_transformer_pbt",
    log_to_file=True,
)

/opt/conda/lib/python3.8/site-packages/ray/tune/tune.py:505: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
  warnings.warn(
(pid=3822550) 2022-10-19 02:38:59.873107: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


== Status ==
Current time: 2022-10-19 02:38:58 (running for 00:00:00.15)
Memory usage on this node: 8.5/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/11.46 GiB heap, 0.0/5.73 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr | train_bs/gpu   |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_2ed07_00000 | RUNNING  | 172.17.0.3:3822550 | 0.190536  | 4.74713e-05 |                |            2 |
| _objective_2ed07_00001 | PENDING  |                    | 0.281782  | 2.12275e-05 |                |            5 

(_objective pid=3822550) Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
(_objective pid=3822550) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=3822550) - This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=3822550) Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/robe

== Status ==
Current time: 2022-10-19 02:39:05 (running for 00:00:07.37)
Memory usage on this node: 13.2/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/11.46 GiB heap, 0.0/5.73 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr | train_bs/gpu   |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_2ed07_00000 | RUNNING  | 172.17.0.3:3822550 | 0.190536  | 4.74713e-05 |                |            2 |
| _objective_2ed07_00001 | PENDING  |                    | 0.281782  | 2.12275e-05 |                |            5

 15%|█▍        | 9/62 [00:06<00:35,  1.48it/s]


...

== Status ==
Current time: 2022-10-19 02:56:19 (running for 00:17:20.76)
Memory usage on this node: 13.3/31.1 GiB
PopulationBasedTraining: 5 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/11.46 GiB heap, 0.0/5.73 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 5/5 (1 RUNNING, 4 TERMINATED)
+------------------------+------------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+-------------+---------+----------------------+
| Trial name             | status     | loc                |   w_decay |          lr | train_bs/gpu   |   num_epochs |   eval_f1 |   eval_accuracy |   eval_loss |   epoch |   training_iteration |
|------------------------+------------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+-------------+---------+---------------

(_objective pid=3827416) 
 57%|█████▋    | 71/125 [00:02<00:02, 24.42it/s]
(_objective pid=3827416) 
 59%|█████▉    | 74/125 [00:02<00:02, 24.43it/s]
(_objective pid=3827416) 
 62%|██████▏   | 77/125 [00:03<00:01, 24.41it/s]
(_objective pid=3827416) 
 64%|██████▍   | 80/125 [00:03<00:01, 24.40it/s]
(_objective pid=3827416) 
 66%|██████▋   | 83/125 [00:03<00:01, 24.41it/s]
(_objective pid=3827416) 
 69%|██████▉   | 86/125 [00:03<00:01, 24.40it/s]
(_objective pid=3827416) 
 71%|███████   | 89/125 [00:03<00:01, 24.39it/s]
(_objective pid=3827416) 
 74%|███████▎  | 92/125 [00:03<00:01, 24.40it/s]
(_objective pid=3827416) 
 76%|███████▌  | 95/125 [00:03<00:01, 24.39it/s]
(_objective pid=3827416) 
 78%|███████▊  | 98/125 [00:03<00:01, 24.40it/s]
(_objective pid=3827416) 
 81%|████████  | 101/125 [00:04<00:00, 24.39it/s]
(_objective pid=3827416) 
 83%|████████▎ | 104/125 [00:04<00:00, 24.37it/s]
(_objective pid=3827416) 
 86%|████████▌ | 107/125 [00:04<00:00, 24.38it/s]
(_objective pid=382741

Result for _objective_2ed07_00002:
  date: 2022-10-19_02-56-22
  done: false
  episodes_total: 0
  epoch: 4.83
  eval_accuracy: 0.973
  eval_f1: 0.9744075829383886
  eval_loss: 0.09452543407678604
  eval_runtime: 6.4908
  eval_samples_per_second: 154.063
  eval_steps_per_second: 19.258
  experiment_id: aab49e31c6fa43f48990833c86d7c485
  hostname: 3481a8a2ae33
  iterations_since_restore: 3
  node_ip: 172.17.0.3
  objective: 1.9474075829383886
  pid: 3827416
  time_since_restore: 120.05144333839417
  time_this_iter_s: 38.65487337112427
  time_total_s: 362.81162762641907
  timestamp: 1666148182
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 3
  trial_id: 2ed07_00002
  warmup_time: 0.0033295154571533203
  
(_objective pid=3827416) {'eval_loss': 0.09452543407678604, 'eval_accuracy': 0.973, 'eval_f1': 0.9744075829383886, 'eval_runtime': 6.4908, 'eval_samples_per_second': 154.063, 'eval_steps_per_second': 19.258, 'epoch': 4.83}


                                                 
100%|██████████| 125/125 [00:06<00:00, 24.25it/s]
                                                 
100%|██████████| 155/155 [01:59<00:00,  1.30it/s]
2022-10-19 02:56:26,167	INFO tune.py:758 -- Total run time: 1047.72 seconds (1047.61 seconds for the tuning loop).


Result for _objective_2ed07_00002:
  date: 2022-10-19_02-56-22
  done: true
  episodes_total: 0
  epoch: 4.83
  eval_accuracy: 0.973
  eval_f1: 0.9744075829383886
  eval_loss: 0.09452543407678604
  eval_runtime: 6.4908
  eval_samples_per_second: 154.063
  eval_steps_per_second: 19.258
  experiment_id: aab49e31c6fa43f48990833c86d7c485
  experiment_tag: 2_num_train_epochs=5
  hostname: 3481a8a2ae33
  iterations_since_restore: 3
  node_ip: 172.17.0.3
  objective: 1.9474075829383886
  pid: 3827416
  time_since_restore: 120.05144333839417
  time_this_iter_s: 38.65487337112427
  time_total_s: 362.81162762641907
  timestamp: 1666148182
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 3
  trial_id: 2ed07_00002
  warmup_time: 0.0033295154571533203
  
== Status ==
Current time: 2022-10-19 02:56:26 (running for 00:17:27.61)
Memory usage on this node: 13.3/31.1 GiB
PopulationBasedTraining: 5 checkpoints, 0 perturbs
Resources requested: 0/20 CPUs, 0/1 GPUs, 0.0/11.46 GiB heap

In [27]:
result

BestRun(run_id='2ed07_00002', objective=1.9474075829383886, hyperparameters={'num_train_epochs': 5, 'weight_decay': 0.025440804525314963, 'learning_rate': 2.7455269030181142e-05, 'warmup_ratio': 0.2543007778205497})

In [28]:
for n, v in result.hyperparameters.items():
    setattr(trainer.args, n, v)

In [29]:
# trainer.args

In [30]:
trainer.train()

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--klue--roberta-base/snapshots/67dd433d36ebc66a42c9aaa85abcf8d2620e41d9/pytorch_model.bin
Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequ

Step,Training Loss,Validation Loss,Accuracy,F1
50,No log,0.403812,0.833000,0.825860
100,No log,0.162208,0.951000,0.953378
150,No log,0.094525,0.973000,0.974408


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
/opt/conda/lib/python3.8/site-packages/transformers/trainer.py:1808: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this messa

TrainOutput(global_step=155, training_loss=0.372362789030998, metrics={'train_runtime': 118.8512, 'train_samples_per_second': 42.069, 'train_steps_per_second': 1.304, 'total_flos': 1313450388357120.0, 'train_loss': 0.372362789030998, 'epoch': 4.99})

In [31]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.09378402680158615,
 'eval_accuracy': 0.974,
 'eval_f1': 0.9753787878787878,
 'eval_runtime': 6.0945,
 'eval_samples_per_second': 164.082,
 'eval_steps_per_second': 20.51,
 'epoch': 4.99}

In [32]:
pred = trainer.predict(test_dataset=test_dataset)
pred

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 50000
  Batch size = 8


PredictionOutput(predictions=array([[-1.581 ,  2.074 ],
       [ 0.7646, -0.9473],
       [ 0.2524, -0.322 ],
       ...,
       [-0.495 ,  0.973 ],
       [ 1.955 , -2.412 ],
       [ 1.684 , -2.037 ]], dtype=float16), label_ids=array([1, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.4627728760242462, 'test_accuracy': 0.8405, 'test_f1': 0.845580404685836, 'test_runtime': 262.3031, 'test_samples_per_second': 190.619, 'test_steps_per_second': 23.827})

In [33]:
label_test = list(pred.label_ids)
pred_test = list(map(lambda x: x.index(max(x)), pred.predictions.tolist()))

In [34]:
print(confusion_matrix(label_test, pred_test))

[[20190  4637]
 [ 3338 21835]]


In [35]:
accuracy = accuracy_score(label_test, pred_test)
f1 = f1_score(label_test, pred_test)
recall = recall_score(label_test, pred_test)
precision = precision_score(label_test, pred_test)

print(accuracy)
print(f1)
print(recall)
print(precision)

0.8405
0.845580404685836
0.867397608548842
0.8248337866424902


In [ ]:
# model_path = "test-model"
# trainer.model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

# Reference

https://bo-10000.tistory.com/154  
https://huggingface.co/blog/ray-tune  
https://docs.ray.io/en/latest/tune/examples/pbt_transformers.html  
https://wood-b.github.io/post/a-novices-guide-to-hyperparameter-optimization-at-scale/#schedulers-vs-search-algorithms  
https://docs.ray.io/en/latest/tune/api_docs/search_space.html  
https://docs.ray.io/en/latest/tune/tutorials/tune-advanced-tutorial.html  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html  
https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/  
https://docs.ray.io/en/latest/tune/faq.html  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#population-based-training-tune-schedulers-populationbasedtraining  
https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.hyperparameter_search  
https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#optuna-tune-search-optuna-optunasearch  
https://kyunghyunlim.github.io/nlp/ml_ai/2021/09/22/hugging_face_5.html  